<a href="https://colab.research.google.com/github/meganificent/Language-Style-Transfer/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.	Insert English phrase
a.	Tokenize English phrase --> get POS array of tuple
2.	Translate English phrase to Spanish using Google Translate
3.	Token Spanish phrase
4.	Spaghetti tokenized Spanish phrase --> output array of tuples with Spanish word as the key and pos as the value
5.	Go through each key of POS English phrase and compare to keys of POS Spanish phrases --> move all English object intervals so that they are in the same positions as their translated Spanish word counterparts
6.	Merge rearranged array of objects so that their keys produce one potentially grammatically incoherent English sentence

Special cases: "a" in spanish, "to" for location in English --> brainstorm placeholders for these kinds of things

In [ ]:
import nltk
import numpy

In [ ]:
from __future__ import print_function

In [ ]:
import spaghetti as sp
import nltk
nltk.download('cess_esp')
mytagger = sp.CESSTagger()
mytagger_uni = mytagger.uni
mytagger_bi = mytagger.bi

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.


*** First-time use of cess tagger ***
Training tagger ...
Tagger trained with cess using UnigramTagger and BigramTagger.
Tagger trained with cess_nomwe using UnigramTagger and BigramTagger.


### **Google Translate**

In [1]:
!pip install googletrans==3.1.0a0
import googletrans
print(googletrans.LANGUAGES)
from googletrans import Translator

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [3]:
translator = Translator(service_urls=['translate.googleapis.com'])


In [11]:
result = translator.translate("el perro blanco corrió", dest="en")
print(result.text)
print(result.src)
print(result.dest)


the white dog ran
es
en


### **English Data**

In [ ]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
Sentence = "The white dog ran"
tokenized = nltk.word_tokenize(Sentence)
print(tokenized)
tagged = nltk.pos_tag(tokenized)
print(tagged)

['The', 'white', 'dog', 'ran']
[('The', 'DT'), ('white', 'JJ'), ('dog', 'NN'), ('ran', 'VBD')]


In [ ]:
Sentence = "white the ran dog."
tokenized = nltk.word_tokenize(Sentence)
print(tokenized)
tagged = nltk.pos_tag(tokenized)
print(tagged)

['white', 'the', 'ran', 'dog', '.']
[('white', 'JJ'), ('the', 'DT'), ('ran', 'NN'), ('dog', 'NN'), ('.', '.')]


### **Spanish Data?**

In [ ]:
!git clone https://github.com/alvations/spaghetti-tagger.git

fatal: destination path 'spaghetti-tagger' already exists and is not an empty directory.


In [5]:
from google.colab import drive
drive.mount('/cont
ent/drive')

Mounted at /content/drive


In [ ]:
nltk.corpus.cess_esp.words()

['El', 'grupo', 'estatal', 'Electricité_de_France', ...]

In [ ]:
nltk.corpus.cess_esp.sents()

[['El', 'grupo', 'estatal', 'Electricité_de_France', '-Fpa-', 'EDF', '-Fpt-', 'anunció', 'hoy', ',', 'jueves', ',', 'la', 'compra', 'del', '51_por_ciento', 'de', 'la', 'empresa', 'mexicana', 'Electricidad_Águila_de_Altamira', '-Fpa-', 'EAA', '-Fpt-', ',', 'creada', 'por', 'el', 'japonés', 'Mitsubishi_Corporation', 'para', 'poner_en_marcha', 'una', 'central', 'de', 'gas', 'de', '495', 'megavatios', '.'], ['Una', 'portavoz', 'de', 'EDF', 'explicó', 'a', 'EFE', 'que', 'el', 'proyecto', 'para', 'la', 'construcción', 'de', 'Altamira_2', ',', 'al', 'norte', 'de', 'Tampico', ',', 'prevé', 'la', 'utilización', 'de', 'gas', 'natural', 'como', 'combustible', 'principal', 'en', 'una', 'central', 'de', 'ciclo', 'combinado', 'que', 'debe', 'empezar', 'a', 'funcionar', 'en', 'mayo_del_2002', '.'], ...]

In [ ]:
nltk.download("cess_esp")

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.


True

In [ ]:
from nltk.corpus import cess_esp as cess
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt

In [ ]:
# Read the corpus into a list, 
# each entry in the list is one sentence.
cess_sents = cess.tagged_sents()

In [ ]:
# Train the unigram tagger
uni_tag = ut(cess_sents)

In [ ]:
spanishSentence = "el perro blanco corrió"

In [ ]:
tokenized = spanishSentence.split()
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)
tagged2 = mytagger_uni.tag(tokenized)
print(tagged2)
tagged4 = mytagger_bi.tag(tokenized)
print(tagged4)
print(tagged4[0])
print(tagged4[0][0]) #How to parce through intervals of the tuple in the array

['el', 'perro', 'blanco', 'corrió']
[('el', 'da0ms0'), ('perro', 'ncms000'), ('blanco', 'aq0ms0'), ('corrió', 'vmis3s0')]
[('el', 'da0ms0'), ('perro', 'ncms000'), ('blanco', 'aq0ms0'), ('corrió', 'vmis3s0')]
[('el', 'da0ms0'), ('perro', 'ncms000'), ('blanco', 'aq0ms0'), ('corrió', 'vmis3s0')]
('el', 'da0ms0')
el


In [ ]:
femSentence = "la niña es fea"
tokenized1 = femSentence.split()
print(tokenized1)
tagged3 = sp.pos_tag(tokenized1)
print(tagged3)

['la', 'niña', 'es', 'fea']
[('la', 'da0fs0'), ('niña', 'ncfs000'), ('es', 'vsip3s0'), ('fea', 'aq0fs0')]


In [ ]:
# Split corpus into training and testing set.
train = int(len(cess_sents)*90/100) # 90%

# Train a bigram tagger with only training data.
bi_tag = bt(cess_sents[:train], backoff=uni_tag)

# Evaluates on testing data remaining 10%
bi_tag.evaluate(cess_sents[train+1:])

# Using the tagger.
bi_tag.tag(Sentence.split(" "))

[('el', 'da0ms0'),
 ('perro', 'ncms000'),
 ('blanco', 'aq0ms0'),
 ('corrió', 'vmis3s0')]

In [ ]:
nltk.corpus.genesis.tagged_words()

AttributeError: ignored

# **Sanskrit Data**

Mounting Drive and importing text files of Sanskrit texts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gItA = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/bhagavadgItA.txt","r")
meghadhUta = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/meghadUta.txt","r")
rAmAyaNa = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/rAmAyaNa.txt", "r")

SyntaxError: ignored

comment to delete later: trying to figure out how to use repository here : https://github.com/ad2476/pos-research which has a sanskrit tagger. 